In [1]:
#Prior to execute this notebook, please make sure you have already cloned the sample data
#!git clone https://github.com/xieyongliang/all-in-one-ai-sample-data.git -t ../../..

In [2]:
import os
import sagemaker
from sagemaker.pytorch import PyTorch

In [3]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
!rm -rf sourcedir.tar.gz
!tar czvf sourcedir.tar.gz *

In [ ]:
source_dir = 's3://{0}/{1}/source/'.format(bucket, 'gabsa')
!aws s3 cp sourcedir.tar.gz $source_dir
source_dir = '{0}sourcedir.tar.gz'.format(source_dir)

In [ ]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'keybert')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'keybert')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

In [12]:
model_name = None
model_data = model_data
entry_point = 'inference.py'
framework_version = '1.9.0'
py_version = 'py38'
model_environment = {
    'type': 'sentence-transformer'
}

In [13]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [14]:
endpoint_name = None
instance_type = 'ml.m5.xlarge'
instance_count = 1

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

In [17]:
from os import listdir
from os.path import isfile, join

sample_dir ='../../../all-in-one-ai-sample-data/keybert/inference/sample'
files = [f for f in listdir(sample_dir) if isfile(join(sample_dir, f))]

inputs  = open(join(sample_dir, files[0]), 'r').read()

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(
    {
        'inputs': inputs
    }
)

In [19]:
predictor.delete_endpoint()